In [1]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd

In [3]:
def read_csv(file_path):
  df = pd.read_csv(file_path)
  X = np.array(df["sentence"])
  Y = np.array(df["label"], dtype=int)
  return X, Y

In [4]:
X_train, Y_train = read_csv("/content/drive/MyDrive/Dataset/Emoji_Text_Classification/train.csv")
X_test, Y_test = read_csv("/content/drive/MyDrive/Dataset/Emoji_Text_Classification/test.csv")

In [5]:
def label_to_emoji(label):
  emojies = ["💚", "⚽️", "😍", "😞", "🍴"]
  return emojies[label]

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [60]:
!unzip -q glove.6B -d/content/drive/MyDrive/Dataset/Emoji_Text_Classification/glove.6B

replace /content/drive/MyDrive/Dataset/Emoji_Text_Classification/glove.6B/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [80]:
f_50d = open("/content/drive/MyDrive/Dataset/Emoji_Text_Classification/glove.6B/glove.6B.50d.txt", encoding="utf-8")
f_100d = open("/content/drive/MyDrive/Dataset/Emoji_Text_Classification/glove.6B/glove.6B.100d.txt", encoding="utf-8")
f_200d = open("/content/drive/MyDrive/Dataset/Emoji_Text_Classification/glove.6B/glove.6B.200d.txt", encoding="utf-8")
f_300d = open("/content/drive/MyDrive/Dataset/Emoji_Text_Classification/glove.6B/glove.6B.300d.txt", encoding="utf-8")

In [81]:
word_vectors = {}

for line in f_300d:
  line = line.strip().split()
  word = line[0]
  vector = np.array(line[1:], dtype=np.float64)
  word_vectors[word] = vector

In [82]:
def sentence_to_avg(sentence):
  try:
    sentence = sentence.lower()
    words = sentence.strip().split(" ")

    sum_vectors = np.zeros((300, ))
    for word in words:
      sum_vectors += word_vectors[word]

    avg_vector = sum_vectors / len(words)
    return avg_vector
  except:
    print(sentence)
    return None

In [83]:
X_train_avg = []

for x_train in X_train:
  X_train_avg.append(sentence_to_avg(x_train))

X_train_avg = np.array(X_train_avg)

X_train_avg.shape

(132, 300)

In [84]:
X_test_avg = []

for x_test in X_test:
  X_test_avg.append(sentence_to_avg(x_test))

X_test_avg = np.array(X_test_avg)

X_test_avg.shape

(56, 300)

In [85]:
Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes=5)
Y_test_one_hot = tf.keras.utils.to_categorical(Y_test, num_classes=5)

In [93]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, input_shape=(300,), activation="softmax")
])

In [94]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [95]:
result_train = model.fit(X_train_avg, Y_train_one_hot, epochs=250)

Epoch 1/250
5/5 [==============================] - 0s 4ms/step - loss: 1.7344 - accuracy: 0.1591
Epoch 2/250
5/5 [==============================] - 0s 3ms/step - loss: 1.6884 - accuracy: 0.1439
Epoch 3/250
5/5 [==============================] - 0s 3ms/step - loss: 1.6434 - accuracy: 0.2121
Epoch 4/250
5/5 [==============================] - 0s 3ms/step - loss: 1.6039 - accuracy: 0.2197
Epoch 5/250
5/5 [==============================] - 0s 3ms/step - loss: 1.6009 - accuracy: 0.2348
Epoch 6/250
5/5 [==============================] - 0s 3ms/step - loss: 1.5402 - accuracy: 0.2803
Epoch 7/250
5/5 [==============================] - 0s 3ms/step - loss: 1.4976 - accuracy: 0.3182
Epoch 8/250
5/5 [==============================] - 0s 3ms/step - loss: 1.5071 - accuracy: 0.3182
Epoch 9/250
5/5 [==============================] - 0s 3ms/step - loss: 1.4798 - accuracy: 0.2652
Epoch 10/250
5/5 [==============================] - 0s 4ms/step - loss: 1.4755 - accuracy: 0.4242
Epoch 11/250
5/5 [===========

In [96]:
result_test = model.evaluate(X_test_avg, Y_test_one_hot)

2/2 [==============================] - 0s 7ms/step - loss: 0.5422 - accuracy: 0.8750


In [97]:
my_test = "I like eating in restaurants"

start = time.time()

my_test_avg = sentence_to_avg(my_test)
my_test_avg = np.array([my_test_avg])

result = model.predict(my_test_avg)

y_pred = np.argmax(result)
print(label_to_emoji(y_pred))
inference_time = (time.time() - start) / 100


1/1 [==============================] - 0s 45ms/step
🍴


In [98]:
print(f"Train loss : ",result_train.history["loss"][249], ",Train accuracy : ",result_train.history["accuracy"][249])
print(f"Test loss : ",result_test[0], ",Test accuracy : ",result_test[1])
print("inference_time : ", inference_time)

Train loss :  0.4049500524997711 ,Train accuracy :  0.9166666865348816
Test loss :  0.5422323942184448 ,Test accuracy :  0.875
inference_time :  0.001113872528076172
